## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Importthe dependencies.
import pandas as pd
import gmaps
import requests
import os
# Import the API key.
from config import g_key 

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Kovernino,RU,57.1253,43.8142,70.12,62,98,4.59,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,69.12,61,100,23.87,overcast clouds
2,2,Rawson,AR,-43.3002,-65.1023,64.02,27,12,11.74,few clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,44.26,70,75,6.91,broken clouds
4,4,Wanning,CN,18.8003,110.3967,80.91,88,100,6.89,overcast clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 120


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Kovernino,RU,57.1253,43.8142,70.12,62,98,4.59,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,69.12,61,100,23.87,overcast clouds
2,2,Rawson,AR,-43.3002,-65.1023,64.02,27,12,11.74,few clouds
4,4,Wanning,CN,18.8003,110.3967,80.91,88,100,6.89,overcast clouds
6,6,Ketchikan,US,55.3422,-131.6461,61.27,82,90,6.91,overcast clouds
8,8,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,64,20,12.66,few clouds
9,9,Fortuna,US,40.5982,-124.1573,63.52,88,1,5.75,clear sky
10,10,Chuy,UY,-33.6971,-53.4616,57.78,72,9,5.61,clear sky
11,11,San Felipe,VE,10.3399,-68.7425,88.84,48,57,7.34,broken clouds
12,12,Kruisfontein,ZA,-34.0033,24.7314,55.89,85,50,20.94,scattered clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                620
City                   620
Country                615
Lat                    620
Lng                    620
Max Temp               620
Humidity               620
Cloudiness             620
Wind Speed             620
Current description    620
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.count()

City_ID                615
City                   615
Country                615
Lat                    615
Lng                    615
Max Temp               615
Humidity               615
Cloudiness             615
Wind Speed             615
Current description    615
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Kovernino,RU,70.12,light rain,57.1253,43.8142,
1,Rikitea,PF,69.12,overcast clouds,-23.1203,-134.9692,
2,Rawson,AR,64.02,few clouds,-43.3002,-65.1023,
4,Wanning,CN,80.91,overcast clouds,18.8003,110.3967,
6,Ketchikan,US,61.27,overcast clouds,55.3422,-131.6461,
8,Arraial Do Cabo,BR,75.16,few clouds,-22.9661,-42.0278,
9,Fortuna,US,63.52,clear sky,40.5982,-124.1573,
10,Chuy,UY,57.78,clear sky,-33.6971,-53.4616,
11,San Felipe,VE,88.84,broken clouds,10.3399,-68.7425,
12,Kruisfontein,ZA,55.89,scattered clouds,-34.0033,24.7314,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [32]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
print(len(clean_hotel_df))

615
558


In [36]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Kovernino,RU,70.12,light rain,57.1253,43.8142,Welcome
1,Rikitea,PF,69.12,overcast clouds,-23.1203,-134.9692,People ThankYou
2,Rawson,AR,64.02,few clouds,-43.3002,-65.1023,Hosteria Sampedro
4,Wanning,CN,80.91,overcast clouds,18.8003,110.3967,Paradise Hotel
6,Ketchikan,US,61.27,overcast clouds,55.3422,-131.6461,Snows Cove Lodge
8,Arraial Do Cabo,BR,75.16,few clouds,-22.9661,-42.0278,Pousada Porto Praia
9,Fortuna,US,63.52,clear sky,40.5982,-124.1573,Super 8 by Wyndham Fortuna
10,Chuy,UY,57.78,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
11,San Felipe,VE,88.84,broken clouds,10.3399,-68.7425,Posada Llovizna Rodríguez
12,Kruisfontein,ZA,55.89,scattered clouds,-34.0033,24.7314,Oyster Bay House Rental


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))